In [ ]:
{-# LANGUAGE BangPatterns, ScopedTypeVariables #-}
import Control.Monad
import Control.Monad.Primitive
import Control.Parallel.Strategies
import Control.Concurrent
import Control.Exception (evaluate)

import qualified Data.Set as Set
import qualified Data.Vector.Unboxed as U

import Numeric.SpecFunctions
import Numeric.MathFunctions.Constants
import Numeric.MathFunctions.Comparison
import Numeric.Polynomial

import Text.Printf(printf)

import IHaskell.Display
import Graphics.Rendering.Chart.Backend.Cairo
import Graphics.Rendering.Chart.Easy hiding (within)

import Debug.Trace

:l NB/Plot
:l NB/Heatmap

greyColormap 0.2
_heat_map_values (def :: HeatMap Double Double Double)

setNumCapabilities 6

In [ ]:
precisionPlot fname a b = do
  xs <- readFile fname
  let vals = map ((\[x,y] -> (read x :: Double, read y :: Double)) . words) $ lines xs
  --
  return $ toRenderable $    
    let isOK x = x > 1e-2
        pts = [ (logBase 10 x, d)
              | (x,y) <- vals
              , let d0 = relativeError y (incompleteBeta a b x)
                    d  = if d0 == 0 then -16.1 else logBase 10 d0
              , not (isNaN d)
              , isOK x
              ]
        ptsBase = [(logBase 10 x,logBase 10 (m_epsilon/2)) 
                  | (x,y) <- vals
                  , isOK x
                  ]
    in layout_plots .~ [ toPlot $ plot_lines_values .~ [pts] $ def
                       , toPlot $ plot_lines_values .~ [ptsBase]
                                $ plot_lines_style .~ (line_color .~ opaque red $ def)
                                $ def
                       ]
      $ layout_title .~ ("A="++show a ++ " B=" ++ show b)
      $ def
      
precisionPlot "../data/ibeta-1-1.dat" 1   1
precisionPlot "../data/ibeta-10-10.dat" 10  10
precisionPlot "../data/ibeta-100-100.dat" 100 100

precisionPlot "../data/ibeta-1000-1000.dat" 1000  1000
precisionPlot "../data/ibeta-4000-4000.dat" 4000 4000

In [ ]:
relativeError (incompleteBeta 100 100 0.01) 1.69065936390433095074273605764064140625905233273204e-142

In [ ]:
relativeError 
  (incompleteBeta 100 100 0.000655868565957143552502817129834511433728039264678955078125)
  2.03987739146908316255079849043021558910893536112196196816367e-260

In [ ]:
relativeError 
  (incompleteBeta 100 100 0.000453582882551019090765914709706407847988884896039962768554688)
  (2.00541346834439406004294904703959964191892215724246345240369e-276)

In [ ]:
incompleteBeta 100 100 0.000453582882551019090765914709706407847988884896039962768554688

In [ ]:
incompleteBeta 100 100 0.000453582882551019090765914709706407847988884896039962768554688

In [ ]:
incompleteBeta 4000 4000 0.3